In [ ]:
#!git clone https://github.com/wchill/HMP_Dataset

# SPARKML PIPELINE

In [ ]:
Clone and create a Spark Dataframe

In [ ]:
fromp pyspark.sql.types import StructType, StructField, IntegerType

schema = StructType([
    StructField('x', IntegerType(), True),
    StructField('y', IntegerType(), True),
    StructField('z', IntegerType(), True)
])

# ETL - Extract Transform Load

In [ ]:
import os

In [ ]:
os.listdir('HMP_Dataset')

In [ ]:
#get rid of folders without underscore which do not have data
file_list_filtered = [s for s in file_list if '_' is s]

In [ ]:
#create Dataframe in Spark
df = None
from pyspark.sql.functions import lit

for category in file_list_filterd:
    data_files= os.listdir('HMP_Dataset/'+ category)
    
    for datafile in data_files:
        print(datafile)
        #creating temporary dataframe
        temp_df = spark.read.option("header", "false").option("delimiter", " ").csv('HMP_Dataset/'+ category+'/'+datafile, schema=schema)
        
        temp_def =temp_def.withColumn('class', lit(category))
        temp_def =temp_def.withColumn('source', lit(datafile))
        
        if df is None:
            df=temp_df
        else:
            #append the new data
            df=df.union(temp_df)

In [ ]:
#Transform class to numbers
# Stringindexer

from pyspark.ml.feature import StringIndexer
# tranform df and assign classindex to class
#creating an estimator in Spark ML, indexer can remember a state
indexer = StringIndexer(imputCol='class', ouputCol= 'classIndex')
indexed = indexer.fit(df).transform(df)

indexed.show()



# one-hot encoding

In [ ]:
from pyspark.ml.feature import OneHotEncoder

#add column for one-hot encodig of class index
encoder=OneHotEncoder(inputCol='classIndex', outputCol ='categoryVec')
encoded = encoder.transform(indexed)

In [ ]:
#call encoded.show should to trigger the Spark job. 
#In Apache Spark are always lazy, if you don't read the data nothing gets executed.
# show() for first 20 rows
# categoryVector is a compressed vector, first element is n.of elments, second is position, third element
encoded.show()

# Feature vectorization

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

#creating vector assembler to transform values x, y, z to vectors, create vectors from ordinary types

vectorAssembler = VectorAssembler(inputCols= ["x", "y", "z"], outputCol= "features")

features_vecotrized = vectorAssembler.transform(encoded)

features_vecotrized.show()

# normalizing

In [ ]:
from pyspark.ml.feature import Normalizer

normalizer = Normalizer(inputCol="features", outputCol="features_norm", p=1.0)
normalized_data = normalizer.transform(features_vecotrized)

##Create the pipeline

In [ ]:
from pyspark.ml import Pipeline

#the pipeline object takes an array of states
pipeline = Pipeline(stages=[indexer, encoder, vectorAssembler, normalizer])

model = pipeline.fit(df)

prediction = model.transform(df)
prediction.show()

In [ ]:
# drop unnecessary columns
df_train = prediction.drop('x').drop('y').drop('z').drop('class').drop('source').drop('features').drop('classIndex')

#returns only categoryVec and features_norm column
df_train.show()